In [1]:
### Monthly and Weekly Time Series

# Below, we will use Facebook Prophet to forecast monthly and weekly spending.

import pandas as pd
from statsmodels.tsa.arima.model import ARIMA
from prophet import Prophet

# read data
df = pd.read_csv('/Users/mac/Documents/Omdena/Building_BankEase/bankease_agentic_app/data/updated_synthetic_bfsi_1000.csv')
# convert the date format
df['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format='%d-%m-%Y %H:%M')

# Run monthly/weekly spending forecasts for each user
def forecast_by_account(df):
    results = []
    for account_id, group in df.groupby('AccountID'):
        # Extract only expenditure transactions
        debits = group[group['TransactionType'] == 'Debit'].copy()
        if debits.empty:
            continue
        debits.set_index('TransactionDate', inplace=True)
        # Monthly/weekly total expenses
        monthly = debits['TransactionAmount'].resample('ME').sum().rename('y')
        weekly  = debits['TransactionAmount'].resample('W-MON').sum().rename('y')
        # Check the number of data items
        if len(monthly) < 1 or len(weekly) < 1:
            continue
                
        # Prophet
        df_mon = monthly.reset_index().rename(columns={'TransactionDate':'ds'})
        m_mon = Prophet(yearly_seasonality=True, weekly_seasonality=False)
        m_mon.fit(df_mon)
        fut_mon = m_mon.make_future_dataframe(periods=1, freq='ME')
        p_mon_fc = m_mon.predict(fut_mon).iloc[-1]['yhat']

        df_wk = weekly.reset_index().rename(columns={'TransactionDate':'ds'})
        m_wk = Prophet(yearly_seasonality=False, weekly_seasonality=True)
        m_wk.fit(df_wk)
        fut_wk = m_wk.make_future_dataframe(periods=1, freq='W-MON')
        p_wk_fc = m_wk.predict(fut_wk).iloc[-1]['yhat']

        results.append({
            'AccountID': account_id,
            'Prophet_month': p_mon_fc,
            'Prophet_week': p_wk_fc
        })
    return pd.DataFrame(results)

# execution
forecast_df = forecast_by_account(df)
print(forecast_df)

/opt/anaconda3/envs/chatbot311/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.
20:28:10 - cmdstanpy - INFO - Chain [1] start processing
20:28:11 - cmdstanpy - INFO - Chain [1] done processing
20:28:11 - cmdstanpy - INFO - Chain [1] start processing
20:28:11 - cmdstanpy - INFO - Chain [1] done processing
20:28:11 - cmdstanpy - INFO - Chain [1] start processing
20:28:11 - cmdstanpy - INFO - Chain [1] done processing
20:28:11 - cmdstanpy - INFO - Chain [1] start processing
20:28:11 - cmdstanpy - INFO - Chain [1] done processing
20:28:11 - cmdstanpy - INFO - Chain [1] start processing
20:28:11 - cmdstanpy - INFO - Chain [1] done processing
20:28:11 - cmdstanpy - INFO - Chain [1] start processing
20:28:11 - cmdstanpy - INFO - Chain [1] done processing

   AccountID  Prophet_month  Prophet_week
0   ACC10000   11243.788389    627.168398
1   ACC10001    4295.524065    729.105219
2   ACC10002   -8018.161019    841.417502
3   ACC10003   -5888.089282    859.889464
4   ACC10004   15347.683985    470.122773
5   ACC10005    9183.986163    415.518227
6   ACC10006   -1290.451376    397.450682
7   ACC10007  -32301.640623    626.183554
8   ACC10008    3414.916989    256.694502
9   ACC10009   13561.662548    439.194646
10  ACC10010    1758.199760    399.962445
11  ACC10011    2659.465998   -102.076738
12  ACC10012  -30476.383439    207.526119
13  ACC10013    2070.615556    424.903770
14  ACC10014   -2769.753130    134.416999
15  ACC10015   -1894.281071    478.578390
16  ACC10016   19439.864344    438.684138
17  ACC10017  -13216.207322    122.369696
18  ACC10018   -6184.687651    623.692789
19  ACC10019   10013.373302    854.501100
20  ACC10020  -35557.373862    526.670989
21  ACC10021   -4299.230159    466.872072
22  ACC10022   34670.812054    369

In [2]:
df.head()


,UserID,TransactionID,AccountID,TransactionAmount,TransactionType,TransactionDate,Merchant,Category,AccountBalance,TransactionDuration,FailedLoginAttempts,Channel,CustomerAge,CustomerOccupation,FraudFlag
0,U001,193a1143-8d0,ACC10000,368.68,Debit,2024-01-01,MediCare,Healthcare,1816.75,107,2,Branch,40,Student,0
1,U001,c40b5a1e-cb8,ACC10000,304.55,Debit,2024-01-04,GeneralStore,Others,1512.20,154,1,ATM,41,Retired,0
2,U001,080e6b8f-f92,ACC10000,99.09,Credit,2024-01-06,NaN,NaN,1611.29,26,1,Online,66,Student,0
3,U001,a039c1b3-c84,ACC10000,32.87,Credit,2024-01-09,NaN,NaN,1644.16,194,2,Branch,68,Retired,0
4,U001,329db876-3df,ACC10000,238.71,Debit,2024-01-12,PharmaStore,Healthcare,1405.45,139,1,Online,56,Engineer,0


In [27]:
filt = (df['UserID'] == 'U001') & (df['TransactionDate'].dt.month == 1)
df_filt = df[filt]
df_filt.head(50)

,UserID,TransactionID,AccountID,TransactionAmount,TransactionType,TransactionDate,Merchant,Category,AccountBalance,TransactionDuration,FailedLoginAttempts,Channel,CustomerAge,CustomerOccupation,FraudFlag
0,U001,193a1143-8d0,ACC10000,368.68,Debit,2024-01-01,MediCare,Healthcare,1816.75,107,2,Branch,40,Student,0
1,U001,c40b5a1e-cb8,ACC10000,304.55,Debit,2024-01-04,GeneralStore,Others,1512.20,154,1,ATM,41,Retired,0
2,U001,080e6b8f-f92,ACC10000,99.09,Credit,2024-01-06,NaN,NaN,1611.29,26,1,Online,66,Student,0
3,U001,a039c1b3-c84,ACC10000,32.87,Credit,2024-01-09,NaN,NaN,1644.16,194,2,Branch,68,Retired,0
4,U001,329db876-3df,ACC10000,238.71,Debit,2024-01-12,PharmaStore,Healthcare,1405.45,139,1,Online,56,Engineer,0
5,U001,66f5e323-a7c,ACC10000,286.01,Debit,2024-01-16,ElectroWorld,Electronics,1119.44,57,2,Branch,24,Retired,0
6,U001,bb4cda91-ad9,ACC10000,94.95,Debit,2024-01-17,QuickBuy,Groceries,1024.49,6,2,ATM,59,Retired,0
7,U001,50620edc-562,ACC10000,422.72,Credit,2024-01-19,NaN,NaN,1447.21,206,2,ATM,48,Retired,0
8,U001,9b677674-1ac,ACC10000,423.82,Debit,2024-01-20,HealthPlus,Healthcare,1023.39,57,2,Online,62,Doctor,0
9,U001,ffbcd1aa-34b,ACC10000,46.53,Debit,2024-01-21,PowerGrid,Utilities,976.86,133,3,Branch,68,Doctor,0


In [13]:
print(df['TransactionDate'].iloc[:3])


0   2024-01-01
1   2024-01-04
2   2024-01-06
Name: TransactionDate, dtype: datetime64[ns]


In [17]:
# read data
df2 = pd.read_csv('/Users/mac/Documents/Omdena/Building_BankEase/bankease_agentic_app/recommendation algo/bank_transactions_data_2.csv')
# convert the date format
df2['TransactionDate'] = pd.to_datetime(df['TransactionDate'], format='%d-%m-%Y %H:%M')
filt2 = (df['UserID'] == 'U001') & (df['TransactionDate'].dt.month == 2)
df_filt2 = df[filt2]
df_filt2.head(50)


,UserID,TransactionID,AccountID,TransactionAmount,TransactionType,TransactionDate,Merchant,Category,AccountBalance,TransactionDuration,FailedLoginAttempts,Channel,CustomerAge,CustomerOccupation,FraudFlag
12,U001,b767584b-c14,ACC10000,68.60,Debit,2024-02-01,FunWorld,Entertainment,463.59,261,1,ATM,31,Student,0
13,U001,70af0476-f7b,ACC10000,64.34,Debit,2024-02-02,FoodieSpot,Dining,399.25,183,3,Branch,66,Retired,0
14,U001,a6b62269-8b7,ACC10000,454.71,Debit,2024-02-03,NetConnect,Utilities,0.00,241,3,Branch,46,Retired,1
15,U001,01df48cd-feb,ACC10000,151.98,Credit,2024-02-04,NaN,NaN,151.98,174,1,ATM,23,Retired,0
16,U001,7ab30536-fc8,ACC10000,214.09,Credit,2024-02-08,NaN,NaN,366.07,132,2,Branch,45,Doctor,0
17,U001,e43d4671-c5a,ACC10000,143.34,Debit,2024-02-11,GoTours,Travel,222.73,125,1,Online,20,Student,0
18,U001,d39408b4-449,ACC10000,349.29,Debit,2024-02-12,GadgetPro,Electronics,0.00,164,1,Online,66,Engineer,0
19,U001,00507b5b-c37,ACC10000,351.19,Credit,2024-02-16,NaN,NaN,351.19,105,1,Online,66,Engineer,0


In [7]:
df.columns

Index(['UserID', 'TransactionID', 'AccountID', 'TransactionAmount',
       'TransactionType', 'TransactionDate', 'Merchant', 'Category',
       'AccountBalance', 'TransactionDuration', 'FailedLoginAttempts',
       'Channel', 'CustomerAge', 'CustomerOccupation', 'FraudFlag'],
      dtype='object')

In [2]:
def generate_recommendations_from_forecast(forecast_df):
    recommendations = []

    for _, row in forecast_df.iterrows():
        acct_id = row['AccountID']
        mon = row['Prophet_month']
        wk = row['Prophet_week']

        recs = []
        if mon > 2000:
            recs.append("Check out our savings advice as we expect higher expenses this month.")
        if wk > 800:
            recs.append("Consider a cash back card if your weekly expenses are high.")
        if mon < 1000 and wk < 300:
            recs.append("With your expenses stable, now may be a good time to start making some short-term investments.")
        if not recs:
            recs.append("Spending patterns are normal. Check out our offers based on your current usage.")

        recommendations.append({
            "AccountID": acct_id,
            "rec_texts": recs
        })
    
    return pd.DataFrame(recommendations)

In [3]:
#!pip install langchain

In [4]:
%pip install langchain-groq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()

llm = ChatGroq(
    temperature=0.3,
    model_name="llama3-8b-8192",
    api_key=os.getenv("GROQ_API_KEY")
)

def generate_agentic_recommendation(account_id, forecast, rec_texts):
    prompt = f"""
You are a financial expert assistant.
The future projected spend for account {account_id} is:：
- Monthly Projected Spending: ${forecast['Prophet_month']:.2f}
- Weekly forecasted spending: ${forecast['Prophet_week']:.2f}

Below are some potential financial advice options：
{chr(10).join(f"- {r}" for r in rec_texts)}

Use this information to generate a sentence of financial advice in natural, familiar language for the user.
"""

    response = llm.invoke(prompt)
    return response.content.strip()


In [6]:
# Step 1: Prediction
forecast_df = forecast_by_account(df)

# Step 2: Rule-based recommendation
rec_df = generate_recommendations_from_forecast(forecast_df)

# Step 3: Natural language generation by the Agent
results = []
for i, row in forecast_df.iterrows():
    acct_id = row['AccountID']
    rec_texts = rec_df[rec_df['AccountID'] == acct_id]['rec_texts'].values[0]
    advice = generate_agentic_recommendation(acct_id, row, rec_texts)
    results.append({
        "AccountID": acct_id,
        "MonthForecast": row['Prophet_month'],
        "WeekForecast": row['Prophet_week'],
        "LLM_Recommendation": advice
    })

recommendation_final_df = pd.DataFrame(results)
print(recommendation_final_df)


21:36:40 - cmdstanpy - INFO - Chain [1] start processing
21:36:40 - cmdstanpy - INFO - Chain [1] done processing
21:36:41 - cmdstanpy - INFO - Chain [1] start processing
21:36:41 - cmdstanpy - INFO - Chain [1] done processing
21:36:41 - cmdstanpy - INFO - Chain [1] start processing
21:36:41 - cmdstanpy - INFO - Chain [1] done processing
21:36:42 - cmdstanpy - INFO - Chain [1] start processing
21:36:42 - cmdstanpy - INFO - Chain [1] done processing
21:36:42 - cmdstanpy - INFO - Chain [1] start processing
21:36:42 - cmdstanpy - INFO - Chain [1] done processing
21:36:42 - cmdstanpy - INFO - Chain [1] start processing
21:36:43 - cmdstanpy - INFO - Chain [1] done processing
21:36:43 - cmdstanpy - INFO - Chain [1] start processing
21:36:43 - cmdstanpy - INFO - Chain [1] done processing
21:36:43 - cmdstanpy - INFO - Chain [1] start processing
21:36:43 - cmdstanpy - INFO - Chain [1] done processing
21:36:43 - cmdstanpy - INFO - Chain [1] start processing
21:36:44 - cmdstanpy - INFO - Chain [1]

   AccountID  MonthForecast  WeekForecast  \
0   ACC10000   22507.886569    627.168398   
1   ACC10001   -4684.470357    729.105219   
2   ACC10002    7396.603099    841.417502   
3   ACC10003    4527.540953    859.889464   
4   ACC10004  -16414.552862    470.122773   
5   ACC10005  -13972.187492    415.518227   
6   ACC10006   -1587.104019    397.450682   
7   ACC10007  -31294.059903    626.183554   
8   ACC10008    4393.259022    256.694502   
9   ACC10009    6800.114496    439.194646   
10  ACC10010   15675.183939    399.962445   
11  ACC10011  -25871.249437   -102.076738   
12  ACC10012    5861.892623    207.526119   
13  ACC10013  -18774.207810    424.903770   
14  ACC10014   19376.913145    134.416999   
15  ACC10015  -20279.099845    478.578390   
16  ACC10016  -12967.023475    438.684138   
17  ACC10017  -15507.868813    122.369696   
18  ACC10018  -28231.251477    623.692789   
19  ACC10019  -13841.854761    854.501100   
20  ACC10020    7412.217579    526.670989   
21  ACC100